## Import Modules

In [16]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

## Open Text File

In [3]:
filename = "sherlock.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

## Character to integer

In [4]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i,c in enumerate(chars))

## Count the number of characters

In [5]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total chars {0}".format(n_chars))
print("Total vocabs {0}".format(n_vocab))

Total chars 561845
Total vocabs 55


## Training data and Labels

In [13]:
seq_length = 100
dataX = []
dataY = []
for i in range(0,n_chars-seq_length,1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total patterns {0}".format(n_patterns))

Total patterns 561745


## One-hot encoding

In [17]:
X = np.reshape(dataX, (n_patterns,seq_length,1))
X = X/float(n_chars)
y = np_utils.to_categorical(dataY)

In [20]:
print(X.shape[1],X.shape[2])

100 1


In [22]:
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1],X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 512)               1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 55)                28215     
Total params: 1,080,887
Trainable params: 1,080,887
Non-trainable params: 0
_________________________________________________________________


In [25]:
filepath = 'weights.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)